In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.2f}'.format)


#API
from fredapi import Fred
import yfinance as yf


#Statistical modules
from statsmodels.tsa.stattools import adfuller
import statsmodels.tsa.filters.bk_filter as fil
from statsmodels.tsa.stattools import kpss
import statsmodels.tsa.stattools as smt

from scipy.stats.stats import pearsonr   
from scipy.stats import variation 



#Modules for simplification of work with data
from collections import defaultdict as col_dict
from tqdm.notebook import tqdm
import warnings
from typing import Union, List, Tuple, TypeVar, Text

PandasDataFrame = TypeVar('pandas.core.frame.DataFrame')



# Loading Data

In [2]:
# сюда вставить свой токен с https://fred.stlouisfed.org/docs/api/api_key.html
# сперва нужно там зарегаться и получить ключ
api_key = '6818673a6dc4586fbb1cd7203eb8a190' 
base_series = 'GDP'
fred = Fred(api_key=api_key)
data = fred.get_series(base_series)
data = pd.DataFrame({'date': data.index, base_series: data.values})
data.tail(5)

,date,GDP
302,2021-07-01,23550.42
303,2021-10-01,24349.12
304,2022-01-01,24740.48
305,2022-04-01,25248.48
306,2022-07-01,25663.29


In [3]:
series_dict = {
#     'GDP': 'Gross Domestic Product', #GDP
    
    "PCE": "Personal Consumption Expenditures", # Consumption
    "GPDI": "Gross Private Domestic Investment", # Investment
    'CBI': 'Total Business Inventories', #Cumulated inventories

    'GDPDEF': 'Gross Domestic Product: Implicit Price Deflator', #Deflator
    'CPIAUCSL': 'Consumer Price Index for All Urban Consumers: All Items in U.S. City Average', #CPI Level
#     'FPCPITOTLZGUSA': 'Inflation, consumer prices for the United States', #CPI inflation
    
    'MSPUS': 'Median Sales Price of Houses Sold for the United States', #Estate prices
    #Stock prices added with yfinance module as snp500                          
    
    "FEDFUNDS": 'Federal Funds Effective Rate', #Short-term nominal rate             
    'IRLTLT01USM156N': 'Long-Term Government Bond Yields: 10-year: Main (Including Benchmark) for the United States', #Long-term nominal
    'TB3SMFFM': '3-Month Treasury Bill Minus Federal Funds Rate', #yield curve
    
    'M1SL': 'M1', #M1
    'MABMM301USM189S': 'M3 for the United States', #M3
    'LOANS':'Loans and Leases in Bank Credit, All Commercial Banks' #total loans
    
}

stock = yf.Ticker("^GSPC")   #stock prices, SNP500
stock = stock.history(period="max")



fred = Fred(api_key=api_key)
for s in series_dict.keys():
    series_data = fred.get_series(s)
    series_data = pd.DataFrame({'date':series_data.index, s:series_data.values})
    data = data.merge(series_data, how = "left", on = "date")
    
data = data.rename({'date':'Date'}, axis='columns')  
data.tail(4)

,Date,GDP,PCE,GPDI,CBI,GDPDEF,CPIAUCSL,MSPUS,FEDFUNDS,IRLTLT01USM156N,TB3SMFFM,M1SL,MABMM301USM189S,LOANS
303,2021-10-01,24349.12,16473.70,4499.17,240.00,121.71,276.59,423600.00,0.08,1.58,-0.03,20063.50,21144000000000.00,10506.68
304,2022-01-01,24740.48,16725.60,4671.03,257.42,124.17,281.93,433100.00,0.08,1.76,0.07,20585.50,21649700000000.00,10783.75
305,2022-04-01,25248.48,17115.60,4609.93,145.37,126.91,288.66,449300.00,0.33,2.75,0.43,20617.60,21644200000000.00,11093.53
306,2022-07-01,25663.29,17398.10,4590.24,100.19,128.18,295.27,454900.00,1.68,2.90,0.58,20532.60,21636200000000.00,11468.25


In [4]:
#Adding stock price

stock = stock['Close']
stock.name = 'Stock'

#filling missing days with previous data because there is no trades in the begging of January

stock = stock.reset_index()
stock['Date'] = stock['Date'].dt.date
stock = stock.set_index('Date')
stock = stock.asfreq('D')

#adding missing days in dataset
stock = stock.fillna(method='ffill')



data = data.set_index('Date').join(stock, lsuffix='Date').reset_index() #creating whole dataset

In [5]:
date_sample = data[
    (data["Date"]>="1970-01-01") & 
    (data["Date"]<="2020-12-31")
#     (data["date"].dt.month == 1)
    ]
data.sample(3)

,Date,GDP,PCE,GPDI,CBI,GDPDEF,CPIAUCSL,MSPUS,FEDFUNDS,IRLTLT01USM156N,TB3SMFFM,M1SL,MABMM301USM189S,LOANS,Stock
200,1996-01-01,7868.47,5085.70,1355.35,6.88,72.69,154.70,137000.00,5.56,5.65,-0.56,1123.50,3647900000000.00,2526.57,615.93
269,2013-04-01,16699.55,11259.30,2775.28,87.76,101.43,231.80,268100.00,0.15,1.76,-0.09,2507.10,10585900000000.00,6973.64,1562.17
297,2020-04-01,19636.73,12082.40,3161.42,-297.61,112.99,256.09,322600.00,0.05,0.66,0.09,4779.80,17002500000000.00,10783.94,2470.50


In [6]:
date_sample = date_sample.reset_index(drop = True)

In [7]:
date_sample['Real GDP'] = np.log(date_sample['GDP']/date_sample['GDPDEF'])
date_sample['Real GPDI'] = np.log(date_sample['GPDI']/date_sample['CPIAUCSL'])
# date_sample['CBI'] = np.log(date_sample['CBI']/date_sample['CPIAUCSL'])
date_sample['Real PCE'] = np.log(date_sample['PCE']/date_sample['CPIAUCSL'])
date_sample['MSPUS'] = np.log(date_sample['MSPUS']/date_sample['CPIAUCSL'])
# date_sample['Stock'] = np.log(date_sample['Stock']/date_sample['CPIAUCSL'])
# date_sample['FEDFUNDS'] = np.log(date_sample['FEDFUNDS']/date_sample['CPIAUCSL'])
# date_sample['IRLTLT01USM156N'] = np.log(date_sample['IRLTLT01USM156N']/date_sample['CPIAUCSL'])
# date_sample['T5YFF'] = np.log(date_sample['T5YFF']/date_sample['CPIAUCSL'])


In [8]:
for i in tqdm(date_sample.columns):
    if i != 'Date':
        date_sample[i] = date_sample[i]/ date_sample[i][0]

  0%|          | 0/18 [00:00<?, ?it/s]

In [9]:
date_sample = date_sample.rename({'GDP':'Nominal GDP',
                                  'Real GDP': 'Real GDP',
                                  
                                  "PCE": 'Nominal Consumption',
                                  'Real PCE': 'Real Consumption',
                                  'GPDI': 'Nominal Investment',
                                  'Real GPDI': 'Real Investment',
                                  'CBI': 'Cumulated inventories',
                                  
                                  'GDPDEF': 'Deflator',
                                  'CPIAUCSL': 'CPI Level',
                                # 'FPCPITOTLZGUSA':'CPI inflation',
                                  
                                  'Stock':'Stock prices',
                                  'MSPUS': 'Real Estate prices',
                                  
                                  "FEDFUNDS": 'Short-term nominal rate',
                                  'IRLTLT01USM156N': 'Long-term nominal',
                                  'TB3SMFFM': 'Yield curve',
                                  
                                  'M1SL': 'M1', 
                                  'MABMM301USM189S': 'M3', 
                                  'LOANS':'Total loans' 
                                 
                                 
                                 }, axis='columns')

In [10]:
date_sample.head(3)

,Date,Nominal GDP,Nominal Consumption,Nominal Investment,Cumulated inventories,Deflator,CPI Level,Real Estate prices,Short-term nominal rate,Long-term nominal,Yield curve,M1,M3,Total loans,Stock prices,Real GDP,Real Investment,Real Consumption
0,1970-01-01,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
1,1970-04-01,1.02,1.01,1.02,2.79,1.01,1.02,1.00,0.90,0.95,1.43,1.00,1.00,1.01,0.98,1.00,1.00,1.00
2,1970-07-01,1.03,1.03,1.03,2.79,1.02,1.03,0.99,0.80,0.96,0.68,1.01,1.02,1.03,0.79,1.00,1.01,1.00


# Checking for stationary

In [11]:
def fxn():
    warnings.warn("deprecated", DeprecationWarning)
    
def stationary_check(Data: PandasDataFrame, stationary: str == 'c'): 
    
    if stationary == 'c':
        c = 'constant'
    else:
        c = 'trend'
        
    
    series_kpss = col_dict(list)
    for i in tqdm(Data.columns):
        result = kpss(Data[i].dropna(), regression = stationary) #drop Nan to kpss to work

                                                       #Checking for stationary around trend|constant.
                                                       # H0: Data is stationary around some trend|constant
                                                       # * - signiﬁcant (or stationary in our case) on 1% level,
                                                       #** - on 5%, *** - 10%

        if result[1] < 0.01:
            series_kpss['Stationary***'].append(i)
        elif result[1] >= 0.01 and result[1] < 0.05:
            series_kpss['Stationary**'].append(i)
        elif result[1] >= 0.05 and result[1] < 0.01:
            series_kpss['Stationary*'].append(i)
        else:
            series_kpss['Not stationary'].append(i)
        
    
        
    print(f'\n \033[1m Around {c}: \033[0m')
    for i in series_kpss:
        print(f'\n \033[1m Amount of {i} time series:\033[0m {len(series_kpss[i])} из {len(date_sample.columns)} .\n\n\033[1m {i} is: \033[0m')
        for i in series_kpss[i]:
            print(f' {i}')
        
with warnings.catch_warnings(): #skipping warnings
    warnings.simplefilter("ignore")
    fxn()
    stationary_check(date_sample, stationary = 'ct')


  0%|          | 0/18 [00:00<?, ?it/s]


  Around trend: 

  Amount of Stationary** time series: 13 из 18 .

 Stationary** is: 
 Date
 Nominal GDP
 Nominal Consumption
 Nominal Investment
 Deflator
 CPI Level
 Long-term nominal
 M1
 M3
 Total loans
 Stock prices
 Real GDP
 Real Consumption

  Amount of Not stationary time series: 5 из 18 .

 Not stationary is: 
 Cumulated inventories
 Real Estate prices
 Short-term nominal rate
 Yield curve
 Real Investment


In [12]:
with warnings.catch_warnings(): #skipping warnings
    warnings.simplefilter("ignore")
    fxn()
    stationary_check(date_sample, stationary = 'c')

  0%|          | 0/18 [00:00<?, ?it/s]


  Around constant: 

  Amount of Stationary** time series: 17 из 18 .

 Stationary** is: 
 Date
 Nominal GDP
 Nominal Consumption
 Nominal Investment
 Deflator
 CPI Level
 Real Estate prices
 Short-term nominal rate
 Long-term nominal
 Yield curve
 M1
 M3
 Total loans
 Stock prices
 Real GDP
 Real Investment
 Real Consumption

  Amount of Not stationary time series: 1 из 18 .

 Not stationary is: 
 Cumulated inventories


# Filtering

In [13]:
d = {}
for i in tqdm(date_sample.columns.drop('Date')): 
    d[i] = np.std(fil.bkfilter(date_sample[i], high = 40, K = 8))

  0%|          | 0/17 [00:00<?, ?it/s]

In [14]:
#Calculation cross correlation between lags

d1 = {}
for i in list(date_sample.columns.drop('Date')):

    backwards = smt.ccf(date_sample[i], date_sample['Real GDP'], unbiased=False)[::-1][-5:]
    forwards = smt.ccf(date_sample[i], date_sample['Real GDP'], unbiased=False)[:5]
    ccf_output = np.r_[backwards[:-1], forwards]
    
    d1[i] = ccf_output

C:\Users\ilmgf\AppData\Local\Temp\ipykernel_13572\2708426983.py:6: FutureWarning: the 'unbiased'' keyword is deprecated, use 'adjusted' instead.
  backwards = smt.ccf(date_sample[i], date_sample['Real GDP'], unbiased=False)[::-1][-5:]
C:\Users\ilmgf\AppData\Local\Temp\ipykernel_13572\2708426983.py:7: FutureWarning: the 'unbiased'' keyword is deprecated, use 'adjusted' instead.
  forwards = smt.ccf(date_sample[i], date_sample['Real GDP'], unbiased=False)[:5]


In [15]:
#Recreating table as in article

rgdp = d['Real GDP']
d = pd.DataFrame(d.items(), columns=['Factors','Absolute'])
d['relative/GDP'] = d['Absolute']/rgdp
d = d.set_index('Factors')
d1 = pd.DataFrame.from_dict(d1 , orient='index', columns=['-4','-3','-2','-1','0','1','2','3','4'])
Macro_US =d.join(d1)


In [16]:
Macro_US.round(2)

,Absolute,relative/GDP,-4,-3,-2,-1,0,1,2,3,4
Factors,,,,,,,,,,,
Nominal GDP,0.10,33.76,0.93,0.94,0.95,0.96,0.96,0.96,0.95,0.94,0.93
Nominal Consumption,0.10,35.95,0.93,0.94,0.95,0.95,0.96,0.95,0.95,0.94,0.93
Nominal Investment,0.43,153.60,0.92,0.93,0.94,0.95,0.96,0.95,0.94,0.93,0.92
Cumulated inventories,16.73,5919.84,0.19,0.20,0.21,0.22,0.24,0.22,0.21,0.20,0.19
Deflator,0.01,4.73,0.94,0.95,0.97,0.98,0.99,0.98,0.97,0.95,0.94
CPI Level,0.03,9.47,0.95,0.96,0.97,0.98,0.99,0.98,0.97,0.96,0.95
Real Estate prices,0.00,1.27,0.86,0.88,0.90,0.92,0.93,0.92,0.90,0.88,0.86
Short-term nominal rate,0.12,43.86,-0.70,-0.70,-0.70,-0.71,-0.72,-0.71,-0.70,-0.70,-0.70
Long-term nominal,0.09,31.68,-0.77,-0.77,-0.77,-0.77,-0.77,-0.77,-0.77,-0.77,-0.77
